In [25]:
import casadi as ca
import numpy as np

In [26]:
num_poles = 3

pole_as = [0.05, 0.1, 0.15]
pole_ls = [0.1, 0.2, 0.3]
pole_ms = [0.1, 0.15, 0.2]
pole_ds = [0.01, 0.01, 0.01]
pole_Js = [0.13, 0.12, 0.11]

m_c = 0.5
g = 9.81

In [27]:
opti = ca.Opti()

s = opti.variable()
d_s = opti.variable()
dd_s = opti.variable()
thetas = [opti.variable() for i in range(num_poles)]
d_thetas = [opti.variable() for i in range(num_poles)]
dd_thetas = [opti.variable() for i in range(num_poles)]
u = dd_s

derivate_by_time_dict = {s: d_s, d_s: dd_s}
for i in range(num_poles):
    derivate_by_time_dict[thetas[i]] = d_thetas[i]
    derivate_by_time_dict[d_thetas[i]] = dd_thetas[i]

def derivate_by_time(expr):
    for var, der in list(derivate_by_time_dict.items())[::-1]:
        expr = ca.substitute(expr, var, der)
    return expr

A = s + thetas[0]
B = d_s + d_thetas[0]
print("A:", A)
print("B:", B)
print("dA:", derivate_by_time(A))

A: (opti10_x_1+opti10_x_4)
B: (opti10_x_2+opti10_x_7)
dA: (opti10_x_2+opti10_x_7)


In [28]:
pole_pcs = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*ca.sin(prev_theta)
        prev_2 += prev_l*ca.cos(prev_theta)
    pole_pcs.append([s-a*ca.sin(theta)+prev_1, a*ca.cos(theta)+prev_2])

In [29]:
def square_sum(x: tuple):
    result = 0
    for x_i in x:
        result += x_i**2
    return result

T_c = 1/2*m_c*d_s**2
T_p = 1/2*sum(m*square_sum(pc) for m, pc in zip(pole_ms, pole_pcs))+1/2*sum(J*d_theta**2 for J, d_theta in zip(pole_Js, d_thetas))
T = T_c + T_p
V = 0
for m, pc in zip(pole_ms, pole_pcs):
    V += g*m*pc[0]
R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2
    prev_w = d_theta

In [30]:
L = T-V
eqs = []
lh = derivate_by_time(ca.jacobian(L, d_s))
lh -= ca.jacobian(L, s) + ca.jacobian(R, d_s)
rh = u
eqs.append(lh-rh)
for theta, d_theta in zip(thetas, d_thetas):
    lh = derivate_by_time(ca.jacobian(L, d_theta))
    lh -= ca.jacobian(L, theta) + ca.jacobian(R, d_theta)
    rh = 0
    eqs.append(lh-rh)
eqs

[MX((((0.25*(2.*opti10_x_3))-(-4.4145+(0.5*(((0.1*(2.*(opti10_x_1-(0.05*sin(opti10_x_4)))))+(0.15*(2.*((opti10_x_1-(0.1*sin(opti10_x_5)))+(-0.1*sin(opti10_x_4))))))+(0.2*(2.*((opti10_x_1-(0.15*sin(opti10_x_6)))+((-0.1*sin(opti10_x_4))+(-0.2*sin(opti10_x_5))))))))))-opti10_x_3)),
 MX(@1=0.05, @2=0.05, @3=(@2*cos(opti10_x_4)), @4=-0.1, @5=(@4*cos(opti10_x_4)), @6=0.1, @7=-0.1, @8=(@7*cos(opti10_x_4)), @9=0.1, ((0.5*(0.13*(2.*opti10_x_10)))-(((0.5*(((0.1*((-((2.*(@1*cos(opti10_x_4)))*(@1*sin(opti10_x_4))))-((2.*(opti10_x_1-(@2*sin(opti10_x_4))))*@3)))+(0.15*(((2.*((opti10_x_1-(0.1*sin(opti10_x_5)))+(@4*sin(opti10_x_4))))*@5)-((2.*((0.1*cos(opti10_x_5))+(@6*cos(opti10_x_4))))*(@6*sin(opti10_x_4))))))+(0.2*(((2.*((opti10_x_1-(0.15*sin(opti10_x_6)))+((@7*sin(opti10_x_4))+(-0.2*sin(opti10_x_5)))))*@8)-((2.*((0.15*cos(opti10_x_6))+((@9*cos(opti10_x_4))+(0.2*cos(opti10_x_5)))))*(@9*sin(opti10_x_4)))))))-(((1.4715*@5)-(0.981*@3))+(1.962*@8)))+((0.005*(2.*opti10_x_7))-(0.005*(2.*(opti10_x_8-opti1

In [31]:
state = np.array([0,0]+[0,0]*num_poles)
control = np.array([0])
opti.subject_to(s == state[0])
opti.subject_to(d_s == state[1])
for i in range(num_poles):
    opti.subject_to(thetas[i] == state[2*i+2])
    opti.subject_to(d_thetas[i] == state[2*i+3])
opti.subject_to(u == control[0])

for eq in eqs:
    opti.subject_to(eq == 0)

opti.minimize(T+V+R)